In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys

sys.path.append('./helpers_models/')
sys.path.append('./data_visualization_and_augmentations/')
sys.path.append('../torch_videovision/')
sys.path.append('./important_csvs/')

from helpers_resnet import *

resnet = torchvision.models.resnet50(pretrained=True)
adaptive_pooling = AdaptiveConcatPool2d()
head = Head()
resnet.avgpool = adaptive_pooling
resnet.fc = head

os.environ['CUDA_VISIBLE_DEVICES']='0' #,1,2

resnet = resnet.cuda()

for param in resnet.parameters():
    param.requires_grad = False
    
for param in resnet.avgpool.parameters():
    param.requires_grad = True
    
for param in resnet.fc.parameters():
    param.requires_grad = True

resnet = nn.DataParallel(resnet)
check_freeze(resnet.module)

In [ ]:
from helpers_thresholds import *

In [ ]:
plt.rcParams['figure.figsize'] = (12,6)
font = {'family' : 'DejaVu Sans',  'weight' : 'normal',  'size'  : 40}
plt.rc('font', **font)

In [ ]:
tensor_transform = get_tensor_transform('ImageNet', False)
train_spat_transform = get_spatial_transform(2)
train_temp_transform = get_temporal_transform()
valid_spat_transform = get_spatial_transform(0)
valid_temp_transform = va.TemporalFit(size=16)

root_dir = '/media/scratch/astamoulakatos/nsea_video_jpegs/'
df = pd.read_csv('./small_dataset_csvs/events_with_number_of_frames_stratified.csv')
# df_train = get_df(df, 20, True, False, False)
# class_image_paths, end_idx, idx_label= get_indices(df_train, root_dir)
# train_loader = get_loader(1, 128, end_idx, class_image_paths, train_temp_transform, train_spat_transform, tensor_transform, False, True, True, 1)
df_valid = get_df(df, 20, False, True, False)
class_image_paths, end_idx, idx_label = get_indices(df_valid, root_dir)
valid_loader = get_loader(1, 50, end_idx, class_image_paths, valid_temp_transform, valid_spat_transform, tensor_transform, False, True, True, 1)
df_test = get_df(df, 20, False, False, True)
class_image_paths, end_idx, idx_label = get_indices(df_test, root_dir)
test_loader = get_loader(1, 50, end_idx, class_image_paths, valid_temp_transform, valid_spat_transform, tensor_transform, False, True, True, 1)

torch.cuda.empty_cache()

In [ ]:
load = True
if load:
    checkpoint = torch.load('/media/scratch/astamoulakatos/saved-resnet-models/fifth-small-aug-final/best-checkpoint-010epoch.pth')
    resnet.load_state_dict(checkpoint['model_state_dict'])
    print('loading pretrained freezed model!')

In [ ]:
device = torch.device('cuda')

In [ ]:
from predictions import *

In [ ]:
from temperature_scaling import *

In [ ]:
scaled_model = ModelWithTemperature(resnet)

In [ ]:
scaled_model.set_temperature(valid_loader)

In [ ]:
scaled_model.model

In [ ]:
scaled_model.model.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for X, y in test_loader:
        X = X.to(device)
        y = Variable(y.float()).to(device) 
        #X = X.permute(0,2,1,3,4)
        y = y.squeeze(dim=1)
        y = y.float()
        output, _ = scaled_model.model(X)
        y = y.detach().cpu()
        #loss = criterion(output, y)
        preds = torch.sigmoid(output)
        preds = preds.to(torch.float32) 
        preds = preds.detach().cpu()
        y_pred.append(preds)
        y_true.append(y)

In [ ]:
y_pred

In [ ]:
y_pr = torch.cat(y_pred)

In [ ]:
y_tr = torch.cat(y_true)

In [ ]:
y_pr = y_pr.numpy()
y_tr = y_tr.numpy()

In [ ]:
y_pr.shape

In [ ]:
from helpers_thresholds import *

In [ ]:
#y_tr = np.vstack([t.__array__() for tensor in y_true for t in tensor])

In [ ]:
#y_pr = np.vstack([t.__array__() for tensor in y_pred for t in tensor])

In [ ]:
classes = ['Exposure', 'Burial', 'Field Joint', 'Anode', 'Free Span']

In [ ]:
plot_pr_curves(0.0, y_tr, y_pr, classes, '_2d_full_size')

In [ ]:
thresholds = nsea_compute_thresholds(y_tr, y_pr, classes)

In [ ]:
thresholds

In [ ]:
df_thresholds = pd.DataFrame({'Threshold': thresholds}).T
df_thresholds

# Table 1

In [ ]:
print(df_thresholds.to_latex(float_format=lambda x: '%.3f' % truncate_decimals(x,3)))

## validation set

In [ ]:
result_valid = new_compute_metrics(y_tr, y_pr, thresholds, classes)

In [ ]:
result_valid

In [ ]:
resnet.eval()
y_true = []
y_pred = []
correct = 0
with torch.no_grad():
    for X, y in valid_loader:
        X = X.to(device)
        y = Variable(y.float()).to(device) 
        #X = X.permute(0,2,1,3,4)
        y = y.squeeze(dim=1)
        y_cl = y
        y = y.float()
        output, _ = resnet(X)
        y = y.detach().cpu().numpy()
        #loss = criterion(output, y)
        preds = torch.sigmoid(output)
        predicted_cl = preds.data > 0.5
        predicted_cl = predicted_cl.to(torch.float32) 
        preds = preds.to(torch.float32) 
        preds = preds.detach().cpu().numpy()
        y_pred.extend(preds)
        y_true.extend(y)
        correct += sum(sum(predicted_cl == y_cl)).item()
        

In [ ]:
sum(sum(predicted_cl == y_cl)).item()

In [ ]:
y_cl.shape, predicted_cl.shape

In [ ]:
from sklearn.calibration import CalibratedClassifierCV, calibration_curve

In [ ]:
preds.shape

In [ ]:
preds = np.array(y_pred).flatten()
labels_oneh = np.array(y_true).flatten()

In [ ]:
preds.shape

In [ ]:
labels_oneh.shape

In [ ]:
len(valid_loader.dataset)

In [ ]:
correct

In [ ]:
len(valid_loader.dataset)*5

In [ ]:
correct_perc = correct / (len(valid_loader.dataset)*5)
print('Accuracy of the network on the validation set images: %d %%' % (100 * correct_perc))
print(correct_perc)

In [ ]:
def calc_bins(preds):
  # Assign each prediction to a bin
    num_bins = 10
    bins = np.linspace(0.1, 1, num_bins)
    binned = np.digitize(preds, bins)

  # Save the accuracy, confidence and size of each bin
    bin_accs = np.zeros(num_bins)
    bin_confs = np.zeros(num_bins)
    bin_sizes = np.zeros(num_bins)

    for bin in range(num_bins):
        bin_sizes[bin] = len(preds[binned == bin])
        if bin_sizes[bin] > 0:
            bin_accs[bin] = (labels_oneh[binned==bin]).sum() / bin_sizes[bin]
            bin_confs[bin] = (preds[binned==bin]).sum() / bin_sizes[bin]
   
    return bins, binned, bin_accs, bin_confs, bin_sizes

In [ ]:
def get_metrics(preds):
    ECE = 0
    MCE = 0
    bins, _, bin_accs, bin_confs, bin_sizes = calc_bins(preds)

    for i in range(len(bins)):
        abs_conf_dif = abs(bin_accs[i] - bin_confs[i])
        ECE += (bin_sizes[i] / sum(bin_sizes)) * abs_conf_dif
        MCE = max(MCE, abs_conf_dif)

    return ECE, MCE

In [ ]:
import matplotlib.patches as mpatches

def draw_reliability_graph(preds):
    ECE, MCE = get_metrics(preds)
    bins, _, bin_accs, _, _ = calc_bins(preds)

    fig = plt.figure(figsize=(8, 8))
    ax = fig.gca()

  # x/y limits
    ax.set_xlim(0, 1.05)
    ax.set_ylim(0, 1)

  # x/y labels
    plt.xlabel('Confidence')
    plt.ylabel('Accuracy')

  # Create grid
    ax.set_axisbelow(True) 
    ax.grid(color='gray', linestyle='dashed')

  # Error bars
    plt.bar(bins, bins,  width=0.1, alpha=0.3, edgecolor='black', color='r', hatch='\\')

  # Draw bars and identity line
    plt.bar(bins, bin_accs, width=0.1, alpha=1, edgecolor='black', color='b')
    plt.plot([0,1],[0,1], '--', color='gray', linewidth=2)

  # Equally spaced axes
    plt.gca().set_aspect('equal', adjustable='box')

  # ECE and MCE legend
    ECE_patch = mpatches.Patch(color='green', label='ECE = {:.2f}%'.format(ECE*100))
    MCE_patch = mpatches.Patch(color='red', label='MCE = {:.2f}%'.format(MCE*100))
    plt.legend(handles=[ECE_patch, MCE_patch])

  #plt.show()
  
    plt.savefig('calibrated_network.png', bbox_inches='tight')

#draw_reliability_graph(preds)

In [ ]:
draw_reliability_graph(preds)

In [ ]:
def T_scaling(logits, args):
    temperature = args.get('temperature', None)
    assert temperature, 'You need to provide the temperature variable in kwargs'
    temperature = temperature.unsqueeze(1).expand(logits.size(0), logits.size(1))
    return logits / temperature

In [ ]:
from focal_loss_2 import *

In [ ]:
temperature = nn.Parameter(torch.ones(1).cuda())
pos_wei = torch.tensor([1, 1, 1.5, 1.5, 1])
pos_wei = pos_wei.cuda()
#criterion = nn.BCEWithLogitsLoss(pos_weight = pos_wei)
criterion = FocalLoss2d(weight=pos_wei,reduction='mean',balance_param=1)
optimizer = optim.LBFGS([temperature], lr=0.001, max_iter=10000, line_search_fn='strong_wolfe')

logits_list = []
labels_list = []

In [ ]:
args = {'temperature': temperature}

In [ ]:
from tqdm import tqdm

In [ ]:
for i, data in enumerate(tqdm(valid_loader, 0)):
    images, labels = data[0].to(device), data[1].to(device)

    resnet.eval()
    with torch.no_grad():
        out, _ = resnet(images)
        logits_list.append(out)
        labels_list.append(labels)

In [ ]:
labels_list = torch.cat(labels_list).to(device)

In [ ]:
# Create tensors
logits_list = torch.cat(logits_list).to(device)

In [ ]:
logits_list

In [ ]:
logits_list.shape

In [ ]:
temps = []
losses = []

In [ ]:
def _eval():
    loss = criterion(T_scaling(logits_list, args), labels_list)
    loss.backward()
    temps.append(temperature.item())
    losses.append(loss)
    return loss


In [ ]:
optimizer.step(_eval)

In [ ]:
print('Final T_scaling factor: {:.2f}'.format(temperature.item()))

plt.subplot(121)
plt.plot(list(range(len(temps))), temps)

plt.subplot(122)
plt.plot(list(range(len(losses))), losses)
plt.show()


In [ ]:
def test(loader, calibration_method=None, **kwargs):
    resnet.eval()
    y_true = []
    y_pred = []
    correct = 0
    y_predss = []
    y_truess = []
    with torch.no_grad():
        for X, y in loader:
            X = X.to(device)
            y = Variable(y.float()).to(device) 
            #X = X.permute(0,2,1,3,4)
            y = y.squeeze(dim=1)
            y_cl = y
            y = y.float()
            output, _ = resnet(X)
            y = y.detach().cpu().numpy()
            #loss = criterion(output, y)
            if calibration_method:
                output = calibration_method(output, kwargs)
                
            preds = torch.sigmoid(output)
            predicted_cl = preds.data > 0.5
            predicted_cl = predicted_cl.to(torch.float32) 
            preds = preds.to(torch.float32) 
            preds = preds.detach().cpu().numpy()
            y_pred.extend(preds)
            y_true.extend(y)
            correct += sum(sum(predicted_cl == y_cl)).item()
            y_predss.append(preds)
            y_truess.append(y)
            
    preds = np.array(y_pred).flatten()
    labels_oneh = np.array(y_true).flatten()
            
    correct_perc = correct / (len(valid_loader.dataset)*5)
    print('Accuracy of the network on the validation set images: %d %%' % (100 * correct_perc))
    print(correct_perc)
    return preds, labels_oneh, y_predss, y_truess

In [ ]:
preds_calibrated, _, y_pred, y_true = test(T_scaling, temperature=temperature)

In [ ]:
draw_reliability_graph(preds_calibrated)

In [ ]:
len(preds_calibrated)

In [ ]:
from helpers_thresholds import *

In [ ]:
y_tr = np.vstack([t.__array__() for tensor in y_true for t in tensor])

In [ ]:
y_pr = np.vstack([t.__array__() for tensor in y_pred for t in tensor])

In [ ]:
classes = ['Exposure', 'Burial', 'Field Joint', 'Anode', 'Free Span']

In [ ]:
plot_pr_curves(0.0, y_tr, y_pr, classes, '_2d_full_size')

In [ ]:
thresholds = nsea_compute_thresholds(y_tr, y_pr, classes)

In [ ]:
thresholds

In [ ]:
df_thresholds = pd.DataFrame({'Threshold': thresholds}).T
df_thresholds

# Table 1

In [ ]:
print(df_thresholds.to_latex(float_format=lambda x: '%.3f' % truncate_decimals(x,3)))

## validation set

In [ ]:
result_valid = new_compute_metrics(y_tr, y_pr, thresholds, classes)

In [ ]:
result_valid

In [ ]:
preds_calibrated, _, y_pred, y_true = test(test_loader, T_scaling, temperature=temperature)

In [ ]:
draw_reliability_graph(preds_calibrated)

In [ ]:
y_tr = np.vstack([t.__array__() for tensor in y_true for t in tensor])

In [ ]:
y_pr = np.vstack([t.__array__() for tensor in y_pred for t in tensor])

In [ ]:
result_test = new_compute_metrics(y_tr, y_pr, thresholds, classes)

In [ ]:
result_test

In [ ]:
print(result_test.to_latex(float_format=lambda x: '%.3f' % truncate_decimals(x,3)))

In [ ]:
precision, recall, f1, _ = precision_recall_fscore_support(y_tr, y_pr)

In [ ]:
fop_uncalibrated, mpv_uncalibrated = calibration_curve(testy, yhat_uncalibrated, n_bins=10, normalize=True)